In [ ]:
import os
import os.path as osp
import time
import math
import torch
import numpy as np

os.chdir('../')

import mmcv    
from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
from mmengine import fileio
from mmdet3d.utils import replace_ceph_backend
from mmengine.structures import InstanceData
from mmdet.models.layers import inverse_sigmoid

In [ ]:
print('torch version:', torch.__version__)

In [ ]:
# cfg = Config.fromfile('my_projects/CMT/configs/cmt_nus.py')
cfg = Config.fromfile('my_projects/CMT/configs/cmt_coda.py')
# cfg = Config.fromfile('my_projects/BEVFusion/configs/bevfusion_lidar-cam_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d.py')
cfg.work_dir = osp.abspath('./test/work_dirs')
runner = Runner.from_cfg(cfg)
# runner.load_checkpoint('models/no_remove_dn.pth')

In [ ]:
runner.model

In [ ]:
for data_batch in runner.test_dataloader:
    break
data_batch = runner.model.data_preprocessor(data_batch, training=False)
batch_inputs_dict = data_batch['inputs']
batch_data_samples = data_batch['data_samples']
imgs = batch_inputs_dict.get('imgs', None)
points = batch_inputs_dict.get('points', None)
img_metas = [item.metainfo for item in batch_data_samples]
gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]

In [5]:
runner.model.pts_bbox_head.bbox_coder

In [ ]:
# 训练过程
runner.model.train()
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

In [ ]:
losses

In [ ]:
# 验证过程
runner.model.eval()
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
        break

In [ ]:
pred = outputs[0].get('pred_instances_3d')
pred['scores_3d'].shape, pred['labels_3d'].shape, pred['bboxes_3d'].shape

In [ ]:
runner.model.train()
with torch.no_grad():
    data_batch = next(iter(runner.train_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('gt_instances_3d')['bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('gt_instances_3d')['labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

In [ ]:
outs['dn_mask_dict']

In [ ]:
runner.model.eval()
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)